In [ ]:
!pip install numpy

In [ ]:
!pip install pandas

In [ ]:
!pip install scikit-learn

In [ ]:
!pip install scikit-image

In [ ]:
!pip install joblib

**Import Required Modules**

In [1]:
import os
from pathlib import Path
import multiprocessing as mp
import numpy as np 
import pandas as pd 
from skimage.io import imread
from sklearn.ensemble import RandomForestClassifier
from subprocess import check_output
from joblib import Parallel, delayed

In [2]:
train_path = "E:\\cameraidentification\\dataset\\train"

**Get Camera Name & File Name**

In [3]:
cameras = os.listdir(train_path)

train_images = []
for camera in cameras:
    for fname in sorted(os.listdir(train_path +'\\'+ camera)):
        train_images.append((camera, fname))

train = pd.DataFrame(train_images, columns=['camera', 'fname'])
print(train.shape)
train.sample(5)

(2750, 2)


,camera,fname
236,HTC-1-M7,(HTC-1-M7)64.jpg
1065,LG-Nexus-5x,(LG5x)68.jpg
1325,Motorola-Droid-Maxx,(MotoMax)54.jpg
145,HTC-1-M7,(HTC-1-M7)23.jpg
454,iPhone-4s,(iP4s)260.jpg


**Read Images Data**

In [4]:
cols = ['a0', 'a1', 'a2', 's0', 's1', 's2']

for col in cols:
    train[col] = None

In [5]:
def color_stats(img_path):
    if type(img_path) is tuple:
        img = imread(train_path + '\\' + img_path[0] + '\\' + img_path[1])
        key = img_path[1]

    # Some images read return info in a 2nd dim. We only want the first dim.
    if img.shape == (2,):
        img = img[0]

    return (key, (img[:, :, 0].mean(), img[:, :, 1].mean(), img[:, :, 2].mean(),
                  img[:, :, 0].std(), img[:, :, 1].std(), img[:, :, 2].std()))

In [6]:
NCORE = 8
color_info = Parallel(n_jobs=NCORE)(
    delayed(color_stats)(i) for i in train_images
)
color_info = dict(color_info)

In [7]:
for n, col in enumerate(cols):
    train[col] = train['fname'].apply(lambda x: color_info[x][n])

In [8]:
train.sample(5)

,camera,fname,a0,a1,a2,s0,s1,s2
2654,Sony-NEX-7,(Nex7)260.JPG,110.409896,110.813988,111.946764,51.204760,50.886280,51.028805
1285,Motorola-Droid-Maxx,(MotoMax)266.jpg,83.146250,84.096297,81.877279,43.626108,40.176873,42.980715
1037,LG-Nexus-5x,(LG5x)42.jpg,153.860048,119.833358,108.141755,46.106483,42.183173,43.510263
1806,Motorola-X,(MotoX)24.jpg,135.957193,111.527499,101.589839,66.723244,74.161544,74.483751
802,iPhone-6,(iP6)79.jpg,174.862643,169.587952,160.104843,50.736644,54.451562,59.006804


In [9]:
y = train['camera'].values
X_train = train[cols].values

**Build RandomForest Model**

In [10]:
clf = RandomForestClassifier(n_estimators=200)
clf.fit(X_train, y)

RandomForestClassifier(n_estimators=200)

**Predict Single Image**

In [12]:
from skimage.io import imread


image_path = 'E:\\cameraidentification\\dataset\\train\\iPhone-6\\(iP6)1.jpg'
image = imread(image_path)
a0 = image[:, :, 0].mean()
a1 = image[:, :, 1].mean()
a2 = image[:, :, 2].mean()
s0 = image[:, :, 0].std()
s1 = image[:, :, 1].std()
s2 = image[:, :, 2].std()
image_features = np.array([[a0, a1, a2, s0, s1, s2]])

predicted_camera = clf.predict(image_features)[0]
print(predicted_camera)

iPhone-6


**Export Model**

In [13]:
import joblib

In [14]:
joblib.dump(clf, 'sourcecameramodel.pkl')

['sourcecameramodel.pkl']